In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

def load_data(base_dir):
    datagen = ImageDataGenerator(rescale=1.0/255.0,
                                 horizontal_flip=True,
                                 vertical_flip=True,
                                 rotation_range=20,
                                 zoom_range=0.2)

    train_gen = datagen.flow_from_directory(
        os.path.join(base_dir, 'train'),
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary',
        classes=['Negative', 'Positive']
    )
    test_gen = datagen.flow_from_directory(
        os.path.join(base_dir, 'test'),
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary',
        classes=['Negative', 'Positive']
    )
    val_gen = datagen.flow_from_directory(
        os.path.join(base_dir, 'val'),
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary',
        classes=['Negative', 'Positive']
    )
    return train_gen, test_gen, val_gen

train_gen, test_gen, val_gen = load_data('/Users/niteshyadav/Lyme_Disease/Lyme Dataset')


Found 4046 images belonging to 2 classes.
Found 507 images belonging to 2 classes.
Found 506 images belonging to 2 classes.


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers.legacy import Adam
from sklearn.metrics import accuracy_score
import random

def create_model(params):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(128, 128, 3)))

    for _ in range(params['num_conv_layers']):
        model.add(layers.Conv2D(params['filters'], (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(params['dense_units'], activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=params['learning_rate']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

def fitness(model, train_gen, val_gen, epochs=5):
    model.fit(train_gen, epochs=epochs, validation_data=val_gen, verbose=0)
    val_loss, val_acc = model.evaluate(val_gen, verbose=0)
    return val_acc

def mutate(params):
    params['num_conv_layers'] = random.choice([1, 2, 3])
    params['filters'] = random.choice([16, 32, 64])
    params['dense_units'] = random.choice([64, 128, 256])
    params['learning_rate'] = random.choice([0.001, 0.0001])
    return params

def crossover(parent1, parent2):
    child = {}
    for key in parent1:
        child[key] = random.choice([parent1[key], parent2[key]])
    return child

def genetic_algorithm(train_gen, val_gen, generations=10, population_size=10):
    population = [{'num_conv_layers': random.choice([1, 2, 3]),
                   'filters': random.choice([16, 32, 64]),
                   'dense_units': random.choice([64, 128, 256]),
                   'learning_rate': random.choice([0.001, 0.0001])}
                  for _ in range(population_size)]
    for generation in range(generations):
        fitness_scores = []
        for params in population:
            model = create_model(params)
            score = fitness(model, train_gen, val_gen)
            fitness_scores.append((score, params))
        fitness_scores.sort(reverse=True, key=lambda x: x[0])
        next_population = [params for _, params in fitness_scores[:population_size//2]]

        while len(next_population) < population_size:
            parent1, parent2 = random.sample(next_population, 2)
            child = mutate(crossover(parent1, parent2))
            next_population.append(child)
        population = next_population

    best_params = fitness_scores[0][1]
    best_model = create_model(best_params)
    return best_model, best_params

best_model, best_params = genetic_algorithm(train_gen, val_gen)



In [4]:
best_model.fit(train_gen, epochs=10, validation_data=val_gen)
test_loss, test_acc = best_model.evaluate(test_gen)
print(f'Test Accuracy: {test_acc}')


Epoch 1/10
127/127 [==============================] - 12s 94ms/step - loss: 0.4975 - accuracy: 0.8141 - val_loss: 0.4875 - val_accuracy: 0.8142
Epoch 2/10
127/127 [==============================] - 12s 91ms/step - loss: 0.4878 - accuracy: 0.8141 - val_loss: 0.4848 - val_accuracy: 0.8142
Epoch 3/10
127/127 [==============================] - 12s 91ms/step - loss: 0.4875 - accuracy: 0.8141 - val_loss: 0.4803 - val_accuracy: 0.8142
Epoch 4/10
127/127 [==============================] - 12s 92ms/step - loss: 0.4833 - accuracy: 0.8141 - val_loss: 0.4745 - val_accuracy: 0.8142
Epoch 5/10
127/127 [==============================] - 12s 92ms/step - loss: 0.4800 - accuracy: 0.8141 - val_loss: 0.4810 - val_accuracy: 0.8142
Epoch 6/10
127/127 [==============================] - 12s 97ms/step - loss: 0.4797 - accuracy: 0.8141 - val_loss: 0.4754 - val_accuracy: 0.8142
Epoch 7/10
127/127 [==============================] - 12s 94ms/step - loss: 0.4736 - accuracy: 0.8144 - val_loss: 0.4661 - val_accuracy: